In [116]:
import json
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer


import pandas as pd


import lyrics data

In [117]:
import_dict = []

with open("all_song_lyrics.json", 'r') as ofile:
    import_dict = json.loads(ofile.read())

 drop duplicate songs that were in the charts more than one year 

In [118]:
import_df = pd.DataFrame(import_dict)

no_dupes_df = import_df.drop_duplicates()
no_dupes_df = no_dupes_df[no_dupes_df['lyrics'].isnull() == False]


assign gender and race to songs (only counts first artist named on song)

In [119]:
# add in the gender/poc

gender_poc_df = pd.read_csv("artists_gender_poc.csv")

no_dupes_df = no_dupes_df.merge(gender_poc_df, on='artist')

export the no dupes to a csv
export the no dupes grouped by artist count to a csv

In [120]:
# export no dupes

no_dupes_df.to_csv("no_dupes_df.csv",index=0)
no_dupes_df.groupby("artist").count().reset_index().to_csv("no_dupes_artist.csv",index=0)

clean the lyrics using regex

In [121]:
no_dupes_dicts = no_dupes_df.to_dict("records")

def remove_bracket_text(text_input):
    string_with_brackets = text_input

    # Define regular expression pattern to match text inside brackets
    pattern = r"\[.*?\]"

    # Remove text inside brackets using re.sub() function
    string_without_brackets = re.sub(pattern, "", string_with_brackets)
    
    return string_without_brackets

# clean non lyrics

for track in no_dupes_dicts:
    try:
        track['lyrics'] = "\n".join(track['lyrics'].split("\n")[1:])
        track['lyrics'] = remove_bracket_text(track['lyrics'])
        track['lyrics'] = re.sub("\d?Embed", "", track['lyrics'])
        track['lyrics'] = re.sub("\n", " ", track['lyrics'])
        track['lyrics'] = re.sub("\'", "", track['lyrics'])
        track['lyrics'] = re.sub("ÄÖ", "", track['lyrics'])
        track['lyrics'] = re.sub("LiveGet tickets as low as", "", track['lyrics'])
        track['lyrics'] = re.sub("\$\d+You might also like", "", track['lyrics'])
        track['lyrics'] = re.sub("See " + track['artist'], "", track['lyrics'])
        
        
        
    except:
        print(track)

function to count words in column `lyrics` of a dataframe 

In [122]:
def count_words(input_df):
    #Defined  your own Stopwords in case the clean data function does not remove all of them
    # stop_words = ['verse','im','get','1000','58','60','80','youre','youve', 'guitar','solo','instrumental','intro','pre',"3","the","and","it","to","in","that","on","of","be"]
    stop_words = "english"
    # count the frequency of each word that aren't on the stop_words lists
    cv = CountVectorizer(stop_words=stop_words, strip_accents='unicode' )
    #Create a dataframe called data_cv to store the the number of times the word was used in  a lyric based their decades
    text_cv = cv.fit_transform(input_df['lyrics'].iloc[:])
    # print(text_cv)
    # print(text_cv.toarray())
    data_cv = pd.DataFrame(text_cv.toarray(),columns=cv.get_feature_names_out())
    print(data_cv.columns)
    # data_cv['decade'] = words_df['decade']
    #created a dataframe that Sums the ocurrence frequency of each word and group the result by decade
    vect_words = data_cv.sum().T
    vect_words = vect_words.reset_index(level=0).rename(columns ={'index':'word'})
    vect_words = vect_words.rename(columns={0:"count"})
    vect_words['portion'] = vect_words['count'] / vect_words['count'].sum()
    #Save the data into a csv file
    #change the order of columns 
    return vect_words.sort_values("count", ascending=0)


one large dataframe of all song lyrics word counts. exports file 

In [123]:
#create the new data frame  with the information of words
words_df = pd.DataFrame(no_dupes_dicts)
counted = count_words(words_df)
counted.to_csv("all-country-words.csv",index=0)

Index(['00', '01', '06', '10', '100', '1021', '109', '12', '12th', '14th',
       ...
       'yеah', 'zag', 'zebco', 'zig', 'zipping', 'zone', 'еnd', 'еnough',
       'еvery', 'еverybody'],
      dtype='object', length=4678)


shows word counts by all top artists

In [124]:
# sort to top artists
sorted_artists = words_df.groupby('artist').count().sort_values("song_name",ascending=0).reset_index()


tracked_artists = []


for artist in list(sorted_artists['artist'])[0:20]:
    if artist in tracked_artists:
        continue
    tracked_artists.append(artist)
    print(artist)
    top_words = count_words(words_df[words_df['artist'] == artist]).head(20)
    print(top_words)

Morgan Wallen
Index(['100', '1021', '21', '30', '409', '4runner', '865', '90', '93',
       'accent',
       ...
       'wrong', 'ya', 'yall', 'yeah', 'yeah1', 'year', 'you1', 'younger',
       'youre', 'youve'],
      dtype='object', length=769)
          word  count   portion
451        ooh     62  0.022278
762       yeah     52  0.018685
309         im     51  0.018326
163       dont     50  0.017966
336       know     45  0.016170
361       like     41  0.014732
17        aint     40  0.014373
193         em     35  0.012576
91       cause     34  0.012217
668    thought     29  0.010420
320       just     26  0.009342
436       need     26  0.009342
393       love     25  0.008983
246        got     25  0.008983
234       girl     25  0.008983
327     killin     24  0.008624
97      chasin     24  0.008624
665      think     24  0.008624
594  somebodys     24  0.008624
735    whiskey     23  0.008264
Luke Combs
Index(['12', '12th', '19th', '40', '65', 'act', 'afraid', 'aint', 'air

Index(['aint', 'air', 'angels', 'answer', 'apart', 'apologize', 'avenue',
       'away', 'baby', 'bank',
       ...
       'wrong', 'yamaha', 'yeah', 'year', 'yes', 'youd', 'youll', 'youre',
       'youve', 'zone'],
      dtype='object', length=350)
           word  count   portion
204         ooh     50  0.052301
342        yeah     33  0.034519
78      drinkin     22  0.023013
172        love     22  0.023013
267  southbound     21  0.021967
164      little     20  0.020921
222      pretty     17  0.017782
201          oh     17  0.017782
275       story     15  0.015690
110       ghost     14  0.014644
300        time     12  0.012552
117       gonna     12  0.012552
141          im     11  0.011506
96     favorite     11  0.011506
161        like     11  0.011506
346       youll     10  0.010460
144        just     10  0.010460
74         dont      9  0.009414
14      believe      9  0.009414
52    christmas      8  0.008368
Lee Brice
Index(['17', '20', '3am', 'act', 'aint', 'angel

opens files of words tagged to categories

In [125]:
categories = {}
with open("word_categories.json",'r') as ofile:
    categories = json.loads(ofile.read())

function to create `all_words_df` which has a column for all parent categories (prefixed `cat_`) and category words. 

In [126]:
def return_all_words_df(words_df):

    all_words_df = pd.DataFrame(words_df)
    all_words_df = all_words_df[all_words_df['lyrics'].isnull()==False]

    all_words_dicts = all_words_df.to_dict('records')

    for track in all_words_dicts:
        matching_df_excerpt = all_words_df[(all_words_df['song_name'] == track['song_name']) & (all_words_df['artist'] == track['artist'])]
        this_song_count = count_words(matching_df_excerpt)
        track['words'] = list(this_song_count['word'])

    for word in categories:
        parent_cat = "cat_" + categories[word]
        for track in all_words_dicts:
            track[word] = word in track['words']
            if parent_cat in track and track[parent_cat] == True:
                continue
            else:
                track[parent_cat] = word in track['words']
                
    return pd.DataFrame(all_words_dicts)


creates `all_words_df` for all songs, exports to csv

In [127]:
all_words_df = return_all_words_df(words_df)
all_words_df.to_csv("all_words_df.csv",index=0)


Index(['ball', 'boots', 'bourbon', 'bout', 'burnin', 'chevy', 'days',
       'different', 'dont', 'dropped', 'fault', 'feet', 'flame', 'gets',
       'gone', 'hard', 'head', 'heart', 'hell', 'honey', 'im', 'ive', 'lean',
       'leanin', 'learnin', 'learning', 'like', 'listen', 'lonely', 'looks',
       'match', 'memory', 'miles', 'money', 'mornin', 'oned', 'owe', 'packed',
       'pew', 'pile', 'pour', 'prayers', 'right', 'sheets', 'sorries', 'spent',
       'strong', 'stuff', 'sure', 'swore', 'thats', 'thinkin', 'time', 'wake',
       'wasted', 'way', 'wind', 'wouldnt', 'yeah', 'you1', 'youre'],
      dtype='object')
Index(['100', '90', 'aint', 'aw', 'away', 'baby', 'bar', 'buzzin', 'cause',
       'cut', 'does', 'doin', 'doing', 'dont', 'doubles', 'drowns', 'em',
       'fade', 'faded', 'feel', 'gone', 'gonna', 'hard', 'hey', 'id', 'im',
       'ive', 'jack', 'just', 'like', 'liquors', 'lookin', 'makes', 'matter',
       'memory', 'mixing', 'money', 'need', 'nothin', 'nothings', 'ou

Index(['aint', 'asked', 'away', 'band', 'barstool', 'born', 'bright', 'brown',
       'buddies', 'burnin', 'car', 'cds', 'comin', 'couple', 'crowd', 'cup',
       'damn', 'day', 'dead', 'deep', 'dodge', 'doin', 'dreams', 'drivin',
       'dues', 'end', 'fame', 'feelin', 'fortune', 'friday', 'glory', 'grand',
       'growing', 'guess', 'guitar', 'guy', 'hand', 'hardwood', 'head',
       'heels', 'hot', 'id', 'im', 'interview', 'jar', 'job', 'just', 'knew',
       'life', 'lightning', 'lights', 'like', 'liquor', 'livin', 'make',
       'makin', 'maybe', 'neon', 'night', 'old', 'ole', 'opry', 'palm',
       'payin', 'rent', 'runnin', 'sayin', 'school', 'singin', 'songs',
       'stage', 'steam', 'tips', 'town', 'true', 'tryin', 'van', 'veins',
       'wasnt', 'werent', 'whered', 'worn'],
      dtype='object')
Index(['aint', 'arrow', 'backroad', 'bar', 'beer', 'blinkin', 'block', 'board',
       'broke', 'bunch', 'chock', 'cinder', 'cleaned', 'clock', 'cold', 'damn',
       'dance', 'dart'

Index(['aint', 'alright', 'away', 'better', 'cause', 'comin', 'did', 'dont',
       'dust', 'end', 'far', 'fell', 'foot', 'gettin', 'girl', 'grass',
       'greener', 'guess', 'hard', 'heals', 'heard', 'hey', 'just', 'kill',
       'know', 'leave', 'lie', 'like', 'lose', 'makes', 'maybe', 'meant',
       'met', 'movin', 'mustve', 'new', 'oh', 'pick', 'puttin', 'said', 'say',
       'seen', 'somebody', 'sounds', 'stronger', 'sure', 'time', 'walk',
       'wasnt', 'watch', 'way', 'win', 'world', 'wouldnt', 'yeah', 'youd',
       'youll', 'youre'],
      dtype='object')
Index(['12th', '19th', 'aint', 'air', 'astrodome', 'bar', 'beer', 'biggest',
       'boots', 'bout', 'cause', 'coldest', 'copperhead', 'couple', 'covered',
       'cowboys', 'dance', 'days', 'dirt', 'dont', 'doors', 'downstairs',
       'drink', 'feel', 'floor', 'got', 'hangin', 'home', 'houston', 'just',
       'killer', 'kinda', 'landed', 'like', 'line', 'lonestar', 'make', 'mess',
       'midnight', 'mind', 'missin', 'm

Index(['24', '365', 'aint', 'ayy', 'baby', 'better', 'bit', 'bring', 'changin',
       'chasin', 'climbin', 'come', 'days', 'dont', 'dreams', 'face', 'girl',
       'got', 'just', 'kissin', 'know', 'light', 'lips', 'little', 'lives',
       'lotta', 'love', 'make', 'monopolize', 'moonlight', 'mountains', 'need',
       'night', 'oh', 'place', 'red', 'risе', 'roads', 'rollin', 'room',
       'seeing', 'shore', 'signed', 'sippin', 'skin', 'sleep', 'smile',
       'spinnin', 'stop', 'sun', 'time', 'touch', 'true', 'tryna', 'want',
       'week', 'wine', 'work', 'world', 'yeah', 'youre', 'еvery'],
      dtype='object')
Index(['40', 'aint', 'bag', 'ball', 'beach', 'beam', 'big', 'bit', 'blue',
       'boot', 'button', 'cap', 'city', 'collar', 'concrete', 'dirt',
       'disappear', 'eyes', 'far', 'friday', 'hand', 'high', 'holding', 'holy',
       'hour', 'im', 'jeans', 'jim', 'just', 'kick', 'like', 'little', 'lot',
       'low', 'map', 'middle', 'morning', 'mud', 'night', 'pack', 'pearl',

Index(['aint', 'baby', 'bar', 'better', 'blowin', 'bourbon', 'bumpin', 'burn',
       'come', 'countdown', 'dark', 'deja', 'dont', 'downtown', 'drunk',
       'forever', 'georgia', 'gone', 'hearts', 'hey', 'im', 'kinda', 'kiss',
       'kissin', 'like', 'lonely', 'long', 'minutes', 'mississippi', 'mm',
       'mornin', 'night', 'nothin', 'oh', 'place', 'playin', 'proof', 'right',
       'rings', 'roulette', 'shots', 'shoulder', 'smoke', 'stop', 'swear',
       'tap', 'things', 'time', 'times', 'tipsy', 'turn', 'used', 'vu', 'wait',
       'way', 'whiskey', 'wind', 'yeah', 'youre', 'youve'],
      dtype='object')
Index(['aint', 'baby', 'beautiful', 'bein', 'carpet', 'cause', 'count', 'dawn',
       'differences', 'dont', 'door', 'dusk', 'favorite', 'fight', 'foot',
       'forever', 'girl', 'goin', 'gonna', 'got', 'growin', 'growing', 'hate',
       'hell', 'heres', 'hole', 'ill', 'im', 'jeans', 'just', 'kinda', 'know',
       'leave', 'let', 'life', 'like', 'look', 'love', 'mean', 'min

...

Index(['ain', 'babe', 'beautiful', 'blanc', 'bound', 'caught', 'cause',
       'change', 'cheeks', 'come', 'dimple', 'don', 'dont', 'em', 'eyes',
       'face', 'flush', 'girl', 'god', 'goes', 'gonna', 'got', 'guard', 'hair',
       'hot', 'hotel', 'hour', 'imperfections', 'just', 'know', 'laughing',
       'let', 'lie', 'like', 'lisa', 'little', 'looked', 'love', 'lovin',
       'mad', 'make', 'masterpiece', 'mess', 'mirror', 'mistakes', 'mona',
       'morning', 'movie', 'need', 'open', 'pool', 'ready', 'say', 'security',
       'seen', 'shape', 'shot', 'smile', 'stay', 'taking', 'tell', 'thinking',
       'thousand', 'times', 'told', 'try', 'uh', 've', 'way', 'white', 'ya',
       'yeah', 'youll', 'youre'],
      dtype='object')
Index(['alive', 'band', 'bar', 'beach', 'breath', 'care', 'carnival', 'cause',
       'chatting', 'cold', 'damn', 'dont', 'fall', 'feel', 'feet', 'finally',
       'free', 'god', 'gonna', 'good', 'got', 'hand', 'holdin', 'id', 'kid',
       'kind', 'knowing'

Index(['33', 'aint', 'alright', 'bag', 'beat', 'country', 'dancin', 'denim',
       'dont', 'em', 'feelin', 'fuse', 'goin', 'gold', 'good', 'got', 'hell',
       'hey', 'lets', 'levis', 'life', 'light', 'livin', 'moon', 'night', 'oh',
       'raisin', 'rebel', 'repeat', 'saint', 'shine', 'sinnin', 'solid',
       'takes', 'tango', 'thing', 'time', 'woo', 'yall', 'yell', 'zag', 'zig'],
      dtype='object')
Index(['act', 'alcohol', 'ass', 'baby', 'bid', 'burn', 'chaser', 'come',
       'coming', 'couple', 'dont', 'easy', 'farewell', 'flames', 'foot',
       'friends', 'getting', 'got', 'gotta', 'ground', 'hangover', 'happy',
       'hard', 'heartache', 'hell', 'help', 'hit', 'hour', 'hurt', 'ill',
       'inch', 'lay', 'let', 'like', 'lime', 'look', 'love', 'make', 'miles',
       'muddle', 'old', 'pain', 'perfect', 'plenty', 'pour', 'raise', 'right',
       'salt', 'shaker', 'shoot', 'shot', 'smile', 'straight', 'theres',
       'time', 'tomorrow', 'try', 've', 'water', 'way', 'whatcha

Index(['aint', 'atlanta', 'bag', 'band', 'bar', 'bee', 'best', 'blonde',
       'boots', 'break', 'breakups', 'brighter', 'broke', 'chance', 'couch',
       'cracked', 'cushion', 'dahlonega', 'dance', 'day', 'dial', 'didnt',
       'dive', 'dont', 'double', 'dreamer', 'drink', 'drunk', 'em', 'flat',
       'friend', 'gas', 'gettin', 'girl', 'going', 'good', 'got', 'grass',
       'greener', 'gunnin', 'hear', 'heart', 'heres', 'hittin', 'homesick',
       'honey', 'just', 'kind', 'kinda', 'leaver', 'left', 'lights', 'little',
       'looking', 'love', 'makin', 'making', 'mamas', 'man', 'meet', 'missin',
       'money', 'new', 'night', 'nothings', 'number', 'oh', 'outside',
       'packed', 'peach', 'pie', 'pretty', 'putting', 'raised', 'right',
       'rock', 'roots', 'runnin', 'saves', 'seen', 'singing', 'slice', 'slow',
       'smoke', 'song', 'takes', 'thats', 'things', 'turn', 'ups', 'wanna',
       'way', 'weve', 'worker', 'worst', 'wrong', 'ya', 'yeah', 'youre'],
      dtype='obje

Index(['ah', 'bet', 'bold', 'brown', 'chest', 'comes', 'conscience', 'cool',
       'damn', 'dark', 'diamond', 'distance', 'dress', 'drink', 'dry', 'em',
       'eyes', 'fits', 'fool', 'gets', 'girl', 'gonna', 'hair', 'hazel', 'hes',
       'im', 'just', 'know', 'leave', 'leaves', 'let', 'letter', 'like',
       'love', 'lovin', 'mad', 'makes', 'martinis', 'maybe', 'mind', 'occupy',
       'pearl', 'promiscuous', 'red', 'right', 'room', 'scarlet', 'smart',
       'starts', 'talkin', 'tell', 'thats', 'think', 'time', 'wear', 'west',
       'whos', 'wonder', 'world', 'yeah', 'youre'],
      dtype='object')
Index(['afraid', 'answers', 'away', 'bet', 'bigger', 'broke', 'build', 'car',
       'cause', 'chip', 'conversation', 'cryin', 'dad', 'didnt', 'dont',
       'driveway', 'dying', 'easy', 'fall', 'fear', 'feel', 'getting', 'god',
       'good', 'grow', 'hang', 'happened', 'happy', 'hat', 'hear', 'hey',
       'hole', 'house', 'ill', 'im', 'itll', 'just', 'know', 'learn', 'life',
       

Index(['ain', 'aint', 'baby', 'best', 'boy', 'bring', 'broke', 'cab', 'cause',
       'change', 'close', 'closin', 'closing', 'come', 'don', 'drink', 'em',
       'eyes', 'feel', 'felt', 'fix', 'girl', 'goin', 'got', 'gotta', 'harder',
       'heart', 'hmm', 'home', 'just', 'kind', 'kiss', 'know', 'lay', 'like',
       'lips', 'll', 'loses', 'make', 'makes', 'meet', 'meets', 'midnight',
       'mind', 'miss', 'oh', 'rain', 'remedy', 'somebody', 'street', 'subway',
       'summer', 'thought', 'time', 'train', 'try', 'wanna', 'way', 'weve',
       'wishin', 'wishing', 'yea', 'yeah', 'youre'],
      dtype='object')
Index(['18th', 'ask', 'best', 'bet', 'cardboard', 'cars', 'cassidy', 'caught',
       'change', 'corner', 'countin', 'couple', 'dancin', 'dancing',
       'daughter', 'did', 'dollars', 'dont', 'drive', 'driving', 'emily',
       'fell', 'forgot', 'friend', 'geography', 'gonna', 'got', 'green', 'gym',
       'hand', 'hands', 'head', 'high', 'home', 'ill', 'im', 'interstate',
   

creates `all_words_df` for all FEMALE songs, exports to csv

In [140]:
all_words_df_female = return_all_words_df(words_df[words_df['gender']=='F'])
all_words_df_female.to_csv("all_words_df_female.csv",index=0)


Index(['actually', 'aint', 'baby', 'better', 'burn', 'closer', 'closure',
       'couple', 'dark', 'dont', 'drink', 'drinking', 'em', 'everybody',
       'feeling', 'footprints', 'getting', 'got', 'heart', 'hopeful', 'im',
       'just', 'know', 'lasts', 'left', 'like', 'long', 'looking', 'lose',
       'make', 'maybe', 'memories', 'miss', 'mе', 'new', 'ooh', 'optimistic',
       'proof', 'realistic', 'really', 'replacing', 'rewind', 'right', 'round',
       'says', 'shot', 'sink', 'sip', 'slow', 'space', 'spark', 'stuck',
       'things', 'thinking', 'throwing', 'time', 'tipsy', 'tonight', 'tracing',
       'try', 'trying', 'wanna', 'went', 'whats', 'whеn', 'wishful', 'wont',
       'wrong', 'yeah'],
      dtype='object')
Index(['air', 'album', 'arе', 'asked', 'autumn', 'ball', 'bed', 'bout',
       'break', 'car',
       ...
       'used', 'walk', 'walked', 'week', 'well1', 'wide', 'wind', 'wont',
       'yeah', 'youve'],
      dtype='object', length=127)
Index(['afraid', 'air', 'awa

Index(['aint', 'beat', 'bed', 'believe', 'cause', 'closet', 'didnt', 'dreams',
       'drum', 'ended', 'forgot', 'friends', 'hard', 'heel', 'hid', 'high',
       'hurt', 'id', 'independence', 'kiss', 'kissed', 'like', 'lipstick',
       'lose', 'lost', 'miss', 'old', 'played', 'records', 'red', 'retired',
       'said', 'sheets', 'shoes', 'snare', 'taller', 'thought', 'turn',
       'wanna', 'wanted', 'wear', 'wings', 'yeah'],
      dtype='object')
Index(['amazing', 'baby', 'blood', 'brightest', 'close', 'come', 'crazy',
       'crowd', 'crown', 'did', 'didn', 'epic', 'eyes', 'fade', 'fame',
       'forever', 'gave', 'glory', 'golden', 'grey', 'heartbeats', 'heaven',
       'hell', 'hey', 'high', 'history', 'ill', 'just', 'knew', 'legends',
       'like', 'll', 'lost', 'loving', 'madness', 'magic', 'marker', 'middle',
       'names', 'neon', 'oh', 'permanent', 'question', 'remember', 'stay',
       'story', 'storyyou', 'sun', 'sweat', 'took', 'town', 'tragic', 'water',
       'way', 'w

counts the number of songs in each parent category and category-specific word. exports to a csv

In [129]:
cat_counts = []

for cat in [x for x in all_words_df.columns if 'cat_' in x] + list(categories.keys()):
    d = {}
    d['word'] = cat
    try:
        d['parent'] = categories[cat]
    except:
        d['parent'] = cat
    d['count'] = all_words_df[cat].sum()
    d['portion'] = d['count'] / len(all_words_df)
    cat_counts.append(d)

pd.DataFrame(cat_counts).to_csv("category_counts.csv", index=0)

for females: counts the number of songs in each parent category and category-specific word. exports to a csv

In [130]:
cat_counts_female = []

for cat in [x for x in all_words_df_female.columns if 'cat_' in x] + list(categories.keys()):
    d = {}
    d['word'] = cat
    try:
        d['parent'] = categories[cat]
    except:
        d['parent'] = cat
    d['count'] = all_words_df_female[cat].sum()
    d['portion'] = d['count'] / len(all_words_df_female)
    cat_counts_female.append(d)

pd.DataFrame(cat_counts_female).to_csv("category_counts_female.csv", index=0)

cell to look up if a word is in a song

In [132]:
[x['song_name'] for x in all_words_df.to_dict('records') if 'bottle' in x['words']]

['865',
 'Ghost Story',
 'Drinking Alone',
 'Handle On You',
 'Hard To Forget',
 'Whiskey And Rain',
 'Hole In The Bottle',
 'Waves',
 'Yours',
 'Dear Alcohol',
 'Tequila',
 'Good Time',
 'I Love My Country',
 'Talk You Out Of It',
 'Get Along',
 'For The First Time',
 'Champagne Night',
 'Some Girls',
 'Dive Bar',
 "What Could've Been",
 'No I In Beer',
 'Hell Right',
 'Burn Out',
 'Day Drunk']

cell to look up if what songs do/don't match a given category

In [133]:
[x['song_name'] for x in all_words_df.to_dict('records') if x['cat_alcohol']==False]

['Cover Me Up',
 "'Til You Can't",
 "Y'all Life",
 'The Kind Of Love We Make',
 'Tomorrow Me',
 "Even Though I'm Leaving",
 'Six Feet Apart',
 'She Got The Best Of Me',
 "Must've Never Met You",
 'Something In The Orange',
 'Fall In Love',
 'Where It Ends',
 'Buy Dirt',
 "If I Didn't Love You",
 "All Too Well (Taylor's Version)",
 "Mr. Perfectly Fine (Taylor's Version) (From The Vault)",
 'Take My Name',
 'Pick Me Up',
 'I Hope',
 'The Good Ones',
 'Damn Strait',
 'In Between',
 'This Is It',
 'Five More Minutes',
 'Got What I Got',
 'We Back',
 'Rearview Town',
 'You Make It Easy',
 'Joy Of My Life',
 'Cold',
 'Starting Over',
 'Millionaire',
 'Broken Halos',
 'Like I Love Country Music',
 'Leave You Alone',
 'Homesick',
 'Good As You',
 'Heaven',
 'Circles Around This Town',
 'The Bones',
 'To Hell & Back',
 'GIRL',
 'Rich',
 'Never Say Never',
 "Ain't Always The Cowboy",
 'Night Shift',
 'Slow Down Summer',
 'Life Changes',
 'Settling Down',
 'Hell Of A View',
 'Monsters',
 'Stick T

check how many songs EITHER mention body parts, alcohol, vehicles or clothing

In [137]:
focus_cat_cols = ['cat_body parts', "cat_alcohol", "cat_vehicle", "cat_clothes"]

all_words_df['focus_topics'] = False
for col in focus_cat_cols:
    all_words_df.loc[all_words_df['focus_topics']==False,'focus_topics'] = all_words_df[col]
all_words_df['focus_topics'].sum()

358